In [1]:
from SmileGAN.Smile_GAN_clustering import single_model_clustering, cross_validated_clustering, clustering_result
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\drizzle\anaconda3\envs\deepcare\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d1_2 = pd.read_csv('../tadpole/TADPOLE_D1_D2.csv')

C:\Users\drizzle\AppData\Local\Temp\ipykernel_9588\4265215034.py:1: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,7

In [3]:
dx_id_dict = {}
for i in d1_2['DX_bl'].unique():
    if i == 'CN':
        dx_id_dict[i] = 1
    else:
        dx_id_dict[i] = -1

In [4]:
roi_list = sorted(list(filter(lambda x: x.startswith('ST'),list(d1_2.columns))))[:-5]

In [5]:
roi_type = list(set([x.split('_')[0][-2:] for x in roi_list]))

In [6]:
print(roi_type)

['SA', 'TS', 'CV', 'TA', 'SV']


In [7]:
TA_list = [x for x in roi_list if x.split('_')[0][-2:] == 'TA']
SA_list = [x for x in roi_list if x.split('_')[0][-2:] == 'SA']
TS_list = [x for x in roi_list if x.split('_')[0][-2:] == 'TS']
SV_list = [x for x in roi_list if x.split('_')[0][-2:] == 'SV']
CV_list = [x for x in roi_list if x.split('_')[0][-2:] == 'CV']

In [8]:
def get_type_df(df,roi_list,dx_id_dict):
    clms = ['RID','DX_bl','AGE']
    clms += roi_list
    data = df[clms].rename(columns={"RID": "participant_id", "DX_bl": "diagnosis"})
    data = data.replace(' ',np.nan)
    data['diagnosis'] = data['diagnosis'].map(dx_id_dict)
    data.dropna(axis=1,how='all',inplace=True)
    data = data.dropna(axis=1,thresh = 7650).apply(pd.to_numeric)
    data = data.groupby('participant_id').first().reset_index()
    train_id,test_id = train_test_split(data['participant_id'],test_size=0.2,random_state=42)
    train = data[data['participant_id'].isin(train_id)]
    train = train.apply(lambda x: x.fillna(x.mean()),axis=0)
    test = data[data['participant_id'].isin(test_id)]
    test = test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return train, test

In [9]:
TA_train,TA_test = get_type_df(d1_2, TA_list, dx_id_dict)

In [10]:
len(TA_train),len(TA_train['participant_id'].unique())

(1389, 1389)

In [11]:
len(TA_train)/20

69.45

In [12]:
len(TA_test)/20

17.4

In [13]:
output_dir = "../output/"
ncluster = 3
start_saving_epoch = 9000
max_epoch = 14000

## three parameters for stopping threshold
WD = 0.10
AQ = 20
cluster_loss = 0.0015

## one parameter for consensus method
consensus_type = "highest_matching_clustering"

In [18]:
output_dir = "../output/"
ncluster = 3
start_saving_epoch = 9000
max_epoch = 14000

In [19]:
## three parameters for stopping threshold
# WD = 0.10
# AQ = 20
# cluster_loss = 0.0015
# batch_size = 500
# WD = 1
# AQ = 20
# cluster_loss = 5
WD = 0.1
AQ = 70
cluster_loss = 0.002
batch_size = 70
## one parameter for consensus method
consensus_type = "highest_matching_clustering"

In [14]:
fold_number = 10  # number of folds the leave-out cv runs
data_fraction = 0.8 # fraction of data used in each fold
cross_validated_clustering(TA_train, ncluster, fold_number, data_fraction, start_saving_epoch, max_epoch,
                           output_dir, WD, AQ, cluster_loss, consensus_type)

Start Smile-GAN for semi-supervised clustering
****** Starting training of Fold 0 ******


  0%|          | 50/14000 [00:02<10:11, 22.81it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  0%|          | 50/14000 [00:02<10:03, 23.13it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  0%|          | 50/14000 [00:02<10:08, 22.93it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  0%|          | 50/14000 [00:02<10:10, 22.86it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  0%|          | 50/14000 [00:02<10:26, 22.27it/s]
KeyboardInterrupt

